In [2]:
import sys
sys.path.append("..")

from helpers.load_data import *
import pandas as pd
import numpy as np

In [3]:
dfs = load_datafolder("raw")

calls_df = dfs['calls.csv']
reasons_df = dfs['reasons.csv']

/home/tanny/vanml-hacks-2023/notebooks/../helpers/load_data.py:20: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_filepath)


In [4]:
cleaned_calls_df = calls_df.dropna(subset=['CallLength']).drop(columns=["Unnamed: 0"], axis=1)
cleaned_reasons_df = reasons_df.drop(columns=["Unnamed: 0"], axis=1).rename({"callreportnum1": "callreportnum"}, axis=1)

In [5]:
joined_df = cleaned_calls_df.merge(cleaned_reasons_df, on='callreportnum', how='inner')
joined_df

,callreportnum,DateStart,TimeStart,DateEnd,TimeEnd,CallLength,CityName,CallType,ContactMethod,CallerDemographicsGender,CallerDemographicsAgeGroup,CallerDemographicsCallerType,CallerTypeAffected3rdParty,CallerDemographicsInterpretationLanguage,CallerDemographicsInterpretationOtherlanguage,PublicityOtherPublicity,Mainreason,ReasonDetail
0,54694316.0,2018-07-16,1899-12-31 16:42:00,2018-07-16,1899-12-31 16:51:00,9,New Westminster,Assessment and Referral,Telephone,Female,19-54 Adult,Individual,NaN,NaN,NaN,Not Obtained,Abuse,Domestic Violence
1,54694316.0,2018-07-16,1899-12-31 16:42:00,2018-07-16,1899-12-31 16:51:00,9,New Westminster,Assessment and Referral,Telephone,Female,19-54 Adult,Individual,NaN,NaN,NaN,Not Obtained,Housing and Homelessness,Immediate Shelter
2,54694245.0,2018-07-16,1899-12-31 16:39:00,2018-07-16,1899-12-31 17:07:00,28,unknown,Assessment and Referral,Text Message,Unknown,19-54 Adult,Individual,NaN,NaN,NaN,Service Provider,BC Hydro CCF,NaN
3,54693989.0,2018-07-16,1899-12-31 16:27:00,2018-07-16,1899-12-31 16:37:00,10,Winnipeg,Assessment and Referral,Telephone,Female,19-54 Adult,Individual,NaN,NaN,NaN,Service Provider,Housing and Homelessness,Immediate Shelter
4,54693908.0,2018-07-16,1899-12-31 16:23:00,2018-07-16,1899-12-31 16:26:00,3,Burnaby,Assessment and Referral,Telephone,Female,19-54 Adult,Service Provider,NaN,NaN,NaN,Not Obtained,Income & Financial Assistance,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314849,110115194.0,2022-12-07,1899-12-31 15:30:00,2022-12-07,1899-12-31 15:38:00,8,Duncan,Assessment and Referral,Telephone,Female,65+ Senior,Individual,NaN,NaN,NaN,Not Obtained,Advocacy,NaN
314850,110115194.0,2022-12-07,1899-12-31 15:30:00,2022-12-07,1899-12-31 15:38:00,8,Duncan,Assessment and Referral,Telephone,Female,65+ Senior,Individual,NaN,NaN,NaN,Not Obtained,Income & Financial Assistance,Other
314851,110115107.0,2022-12-07,1899-12-31 15:28:00,2022-12-07,1899-12-31 15:35:00,7,Vancouver,Assessment and Referral,Telephone,Male,65+ Senior,Individual,NaN,NaN,NaN,Not Obtained,Housing and Homelessness,Immediate Shelter
314852,110114724.0,2022-12-07,1899-12-31 15:19:00,2022-12-07,1899-12-31 15:21:00,2,Vancouver,Assessment and Referral,Telephone,Male,65+ Senior,Service Provider,NaN,NaN,NaN,Service Provider,Housing and Homelessness,Immediate Shelter


In [6]:
joined_df = joined_df[(joined_df['CityName'] != 'unknown') & (joined_df['CallerDemographicsInterpretationLanguage'] != 'Other') & (joined_df['CallerDemographicsInterpretationLanguage'] != 'Other specify below')]
joined_df['CallerDemographicsInterpretationLanguage'] = joined_df['CallerDemographicsInterpretationLanguage'].copy().fillna('English')

gender_map = {"2 Sprited": "2 Spirit", "Trans Female": "Transgender", "Trans Male": "Transgender"}
joined_df['CallerDemographicsGender'] = joined_df['CallerDemographicsGender'].copy().replace(gender_map)

/tmp/ipykernel_26961/921071275.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_df['CallerDemographicsInterpretationLanguage'] = joined_df['CallerDemographicsInterpretationLanguage'].copy().fillna('English')
/tmp/ipykernel_26961/921071275.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_df['CallerDemographicsGender'] = joined_df['CallerDemographicsGender'].copy().replace(gender_map)


In [7]:
joined_df.to_csv('final_data.csv', index=False)